In [1]:
%matplotlib qt5
import nptdms
import matplotlib.pyplot as plt
import glob

In [2]:
import numpy as np
import scipy.signal


def am_demodulate(
    signal,
    reference,
    modulation_frequency,
    sampling_rate=10000,
    low_cut=15,
    order=5,
):

    """
    demodulates photodetector input to get quadrature and in-phase components
    :param signal:
    :param reference:
    :param modulation_frequency:
    :param sampling_rate:
    :param low_cut:
    :param order:
    :return:
    """
    normalised_reference = reference - reference.mean()
    samples_per_period = sampling_rate / modulation_frequency
    samples_per_quarter_period = round(samples_per_period / 4)

    shift_90_degrees = np.roll(
        normalised_reference, samples_per_quarter_period
    )

    in_phase = signal * normalised_reference
    in_phase_filtered = _apply_butterworth_lowpass_filter(
        in_phase, low_cut_off=low_cut, fs=sampling_rate, order=order
    )

    quadrature = signal * shift_90_degrees
    quadrature_filtered = _apply_butterworth_lowpass_filter(
        quadrature, low_cut_off=low_cut, fs=sampling_rate, order=order
    )

    return quadrature_filtered, in_phase_filtered


def _demodulate_quadrature(quadrature, in_phase):
    return (quadrature ** 2 + in_phase ** 2) ** 0.5


def _apply_butterworth_lowpass_filter(
    demod_signal, low_cut_off=15, fs=10000, order=5
):
    w = low_cut_off / (fs / 2)  # Normalize the frequency
    b, a = scipy.signal.butter(order, w, "low")
    output = scipy.signal.filtfilt(b, a, demod_signal)
    return output


def demodulate(raw, ref_211, ref_531, sampling_rate):
    """
    gets demodulated signals for 211hz and 531hz am modulated signal
    :param raw:
    :param ref_211:
    :param ref_531:
    :return:
    """

    q211, i211 = am_demodulate(raw, ref_211, 211, sampling_rate=sampling_rate)
    q531, i531 = am_demodulate(raw, ref_531, 531, sampling_rate=sampling_rate)
    demodulated_211 = _demodulate_quadrature(q211, i211)
    demodulated_531 = _demodulate_quadrature(q531, i531)

    return demodulated_211, demodulated_531


def lerner_deisseroth_preprocess(
    photodetector_raw_data,
    reference_channel_211hz,
    reference_channel_531hz,
    sampling_rate,
):
    """
    process data according to https://www.ncbi.nlm.nih.gov/pubmed/26232229 , supplement 11
    :param photodetector_raw_data: the raw signal from the photodetector
    :param reference_channel_211hz:  a copy of the reference signal sent to the signal LED (Ca2+ dependent)
    :param reference_channel_531hz:  a copy of the reference signal sent to the background LED (Ca2+ independent)
    :return: deltaF / F
    """
    demodulated_211, demodulated_531 = demodulate(
        photodetector_raw_data,
        reference_channel_211hz,
        reference_channel_531hz,
        sampling_rate,
    )

    signal = _apply_butterworth_lowpass_filter(
        demodulated_211, 2, sampling_rate, order=2
    )
    background = _apply_butterworth_lowpass_filter(
        demodulated_531, 2, sampling_rate, order=2
    )

    regression_params = np.polyfit(background, signal, 1)
    bg_fit = regression_params[0] * background + regression_params[1]

    delta_f = (signal - bg_fit) / bg_fit
    return signal, bg_fit, delta_f

In [14]:
inputDir = "/mnt/winstor/swc/sjones/users/francesca/photometry_2AC/freely_moving_photometry_data/"

AnimalID = 'SNL_photo70'
Date = '20220203'

File = glob.glob(inputDir + AnimalID +'/' + Date + '*/AI.tdms')[0]
Data = nptdms.TdmsFile(File)

refAnimalID = 'SNL_photo70'
refDate = '20220201'

refFile = glob.glob(inputDir + refAnimalID +'/' + refDate + '*/AI.tdms')[0]
refData = nptdms.TdmsFile(refFile)

data_dict = {}

for variable in ['Data', 'refData']:
    data_dict[variable] = eval(variable)

colors = ['blue', 'orange']    
for c, (k, v) in zip(colors, data_dict.items()):
    
    chan_0 = v.group_channels('acq_task')[0].data
    led405 = v.group_channels('acq_task')[2].data
    led465 = v.group_channels('acq_task')[1].data
    clock = v.group_channels('acq_task')[3].data
    stim_trigger = v.group_channels('acq_task')[4].data

    sig, back, df=lerner_deisseroth_preprocess(chan_0[50000:], led465[50000:], led405[50000:], 10000)
    
    plt.ylim([0.02,0.035])
    plt.plot(back[::100], label=k, color=c)
    plt.plot(sig[::100],color=c, alpha=0.4)
    #plt.plot(df[::100], color=c)
    print(k + ' is ' + c)

plt.legend()

/home/matthew/miniconda3/envs/freely_moving_photometry_analysis/lib/python3.9/site-packages/nptdms/tdms.py:1097: UserWarning: 'TdmsFile.group_channels' is deprecated and will be removed in a future release. Use TdmsFile[group_name].channels().
  warnings.warn(message)


Data is blue


/home/matthew/miniconda3/envs/freely_moving_photometry_analysis/lib/python3.9/site-packages/nptdms/tdms.py:1097: UserWarning: 'TdmsFile.group_channels' is deprecated and will be removed in a future release. Use TdmsFile[group_name].channels().
  warnings.warn(message)


refData is orange


In [6]:
a_dict

{'x': 2, 'y': 1}

In [12]:
chan_0 = data.group_channels('acq_task')[0].data
led405 = data.group_channels('acq_task')[2].data
led465 = data.group_channels('acq_task')[1].data
clock = data.group_channels('acq_task')[3].data
stim_trigger = data.group_channels('acq_task')[4].data


/home/matthew/miniconda3/envs/freely_moving_photometry_analysis/lib/python3.9/site-packages/nptdms/tdms.py:1097: UserWarning: 'TdmsFile.group_channels' is deprecated and will be removed in a future release. Use TdmsFile[group_name].channels().
  warnings.warn(message)


In [13]:
sig, back, df=lerner_deisseroth_preprocess(chan_0[50000:], led465[50000:], led405[50000:], 10000)

In [14]:
plt.ylim([0.02,0.035])
plt.plot(back[::100], label='background', color='blue')
plt.plot(sig[::100], label='signal',color='blue', alpha=0.4)
#plt.plot(df[::100], label='delta_f')
plt.legend()

In [53]:
np.pad(df, 50000).shape

(2557500,)

In [10]:
plt.figure()
plt.plot(sig[:40000000])
plt.plot(back[:40000000])

In [54]:
plt.plot(df[:30000000])

In [22]:
df.shape

(38757500,)

In [22]:
demodulated_211, demodulated_531 = demodulate(chan_0, led465, led405, 10000)

signal = _apply_butterworth_lowpass_filter(demodulated_211, 2, order=2)[25000:]
background = _apply_butterworth_lowpass_filter(demodulated_531, 2, order=2)[25000:]

In [23]:
plt.figure()
plt.plot(signal-np.mean(signal))
plt.plot(background- np.mean(background))
plt.ylim([-0.01,0.03])

(-0.01, 0.03)

In [16]:
demod_trace_filename = 'signal.npy'
background_filename = 'background.npy'
df_filename = 'demod_signal.npy'
np.save(folder + demod_trace_filename, signal)
np.save(folder + background_filename, background)
np.save(folder + df_filename, df)

